In [1]:
import pandas as pd
import numpy as np
import datetime as dt

'''
Instructions: youll need to put the exchange rate table "CurrencyTable" into the same folder as your
existing kickstarter file. The code that transforms the data takes forever finish - at least 3 mins.
'''

#change the path that the read_csv is looking at
df = pd.read_csv('CurrencyTable.csv',index_col=0)
df.index = pd.to_datetime(df.index)
new_index = pd.date_range(df.index.min(),df.index.max(),freq='MS')
df=df.reindex(new_index)
df=df.interpolate().unstack()
rate = pd.DataFrame(df)
rate.columns = ['rate']
rate.rename_axis(['currency','rtdate']).head()
#rate = df.interpolate().unstack().rename_axis(['currency','rtdate']).reset_index()

rate
currency rtdate              
AUD      2010-01-01  0.897274
         2010-02-01  0.896608
         2010-03-01  0.895943
         2010-04-01  0.895277
         2010-05-01  0.894612

In [2]:
#Import the data set into dataframe
#Change the path that read_csv is looking at
tdf = pd.read_csv('ksdata.csv', index_col=0)

In [3]:
'''
Converting the dataset
'''

#Convert columns to datetime
tdf.launched = pd.to_datetime(tdf.launched)
tdf.deadline = pd.to_datetime(tdf.deadline)

#Drop the columns where the launch date is 1970 - 5 itmes total
tdf = tdf.drop(tdf[tdf.launched < '2000-01-01'].index)

#create a column that gives you the duration of the campaign in days
tdf['date_diff'] = tdf.deadline - tdf.launched
tdf.date_diff = tdf.date_diff.apply(lambda x: x.days)

#change the country code for N,0" error to first two letters of currency
tdf['country'] = np.where(tdf['country'] == 'N,0"',tdf.currency.str[:2],tdf['country'])

#create new column
tdf['successful'] = tdf.state

#change funding_status to either 'successful' or 'failed' based on pledged > goal
tdf['successful'] = np.where(tdf['usd_pledged_real'] >= tdf['usd_goal_real'], 1 , 0)

#added a new column to dataframe that matches other dataframe for join
tdf['rtdate'] = tdf['launched'].values.astype('datetime64[M]')

#drop usd_goal_real and usd_pledged_real
tdf.drop(['usd_goal_real','usd_pledged_real','usd pledged'], axis=1, inplace=True)


In [4]:
result = tdf.join(rate, on=['currency', 'rtdate'], how='left')

In [5]:
result['goal_usd'] = result.goal * result.rate
result['pledged_usd'] = result.pledged * result.rate

In [183]:
result.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,...,date_diff,successful,rtdate,rate,goal_usd,pledged_usd,name_len,name_low,name_words,number_words
ID,,,,,,,,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,...,58,0,2015-08-01,1.527486,1527.485876,0.0,31,the songs of adelaide & abullah,"[the, songs, of, adelaide, &, abullah]",6
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,...,59,0,2017-09-01,1.000000,30000.000000,2421.0,45,greeting from earth: zgac arts capsule for et,"[greeting, from, earth:, zgac, arts, capsule, for, et]",8
1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,...,44,0,2013-01-01,1.000000,45000.000000,220.0,14,where is hank?,"[where, is, hank?]",3
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,...,29,0,2012-03-01,1.000000,5000.000000,1.0,49,toshicapital rekordz needs help to complete album,"[toshicapital, rekordz, needs, help, to, complete, album]",7
1000011046,Community Film Project: The Art of Neighborhood Filmmaking,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,...,55,0,2015-07-01,1.000000,19500.000000,1283.0,58,community film project: the art of neighborhood filmmaking,"[community, film, project:, the, art, of, neighborhood, filmmaking]",8


# Analysis
1. Keywords

In [36]:
pd.set_option('display.max_colwidth', -1)

In [31]:
index = range(len(result))
values = result.values
columns = result.columns
df = pd.DataFrame(values, index=index, columns = columns)


In [32]:
dfname = df

In [33]:
import re

In [40]:
dfname['name'] = dfname.name.astype(str)

In [43]:
# there are 23105  projects with (canceled) in the name
pattern = '\(canceled\)'
dfname['name'] = dfname.name.str.replace(pattern, '', n=1, case=False)

In [44]:
# there are 1060  projects with (suspended) in the name
pattern = '\(suspended\)'
dfname['name'] = dfname.name.str.replace(pattern, '', n=1, case=False)

In [47]:
pattern = '[^A-Za-z0-9!? ]'
dfname['name'] = dfname.name.str.replace(pattern, '')

In [48]:
pattern = '\(\)'
dfname['name'] = dfname.name.str.replace(pattern, '')

In [67]:
pattern = '  '
dfname['name'] = dfname.name.str.replace(pattern, ' ')
dfname['name'] = dfname.name.str.replace(pattern, ' ')
dfname['name'] = dfname.name.str.replace(pattern, ' ')
dfname['name'] = dfname.name.str.replace(pattern, ' ')
dfname['name'] = dfname.name.str.replace(pattern, ' ')

In [68]:
dfname.name.str.find(pattern).sum()

-378654

In [69]:
dfname['name_len'] = dfname.name.str.len()

In [70]:
dfname['name_low'] = dfname.name.str.lower()

In [72]:
dfname['name_words']  = dfname.name_low.str.strip().str.split('[\s]')

In [73]:
dfname['number_words']  =  dfname.name_words.str.len()

In [99]:
dfname = dfname.drop(dfname[dfname.name == ' '].index, axis=0) 

In [143]:
# http://sigdelta.com/blog/text-analysis-in-pandas/

In [101]:
dfname[['successful', 'number_words']].groupby('successful').number_words.mean()

successful
0    5.173703
1    5.738358
Name: number_words, dtype: float64

In [102]:
dfname[['successful', 'name_len']].groupby('successful').mean()

,name_len
successful,
0,31.836251
1,34.781074


In [103]:
dfname.describe()

,name_len,number_words
count,378650.000000,378650.000000
mean,32.902049,5.378064
std,15.203882,2.592090
min,1.000000,1.000000
25%,20.000000,3.000000
50%,32.000000,5.000000
75%,46.000000,7.000000
max,85.000000,27.000000


In [79]:
dfname.columns

Index(['name', 'category', 'main_category', 'currency', 'deadline', 'goal',
       'launched', 'pledged', 'state', 'backers', 'country', 'date_diff',
       'successful', 'rtdate', 'rate', 'goal_usd', 'pledged_usd', 'name_len',
       'name_low', 'name_words', 'number_words'],
      dtype='object')

In [80]:
pd.set_option('display.width', 1000)

In [81]:
pd.set_option('display.max_colwidth', -1)

In [104]:
dfname.name_words.isnull().sum()

0

In [105]:
table2 = dfname

In [106]:
rows= list()
for row in table2[['successful', 'name_words']].iterrows():
    r = row[1]
    for word in r.name_words:
        rows.append((r.successful, word))
        
words = pd.DataFrame(rows, columns=['successful', 'word'])
words.head()

,successful,word
0,0,the
1,0,songs
2,0,of
3,0,adelaide
4,0,abullah


In [107]:
words[words.word == '']

,successful,word


In [132]:
count = words.pivot_table(index = 'word', columns='successful',aggfunc=np.size)


In [134]:
count = count.sort_values(1, ascending=False)

In [139]:
count['prop_successful'] = count[1] / len(dfname[dfname.successful == 1])

In [142]:
count['prop_failed'] = count[0] / len(dfname[dfname.successful == 0])

In [144]:
count['diff'] = count['prop_successful'] - count['prop_failed']

In [148]:
sorted_bydiff = count.sort_values(by='diff', ascending=False)

In [149]:
sorted_bydiff

successful,0,1,prop_successful,prop_failed,diff
word,,,,,
the,127354.0,82718.0,0.603596,0.527110,0.076486
a,59402.0,42982.0,0.313641,0.245861,0.067780
album,10410.0,13454.0,0.098174,0.043086,0.055088
new,13666.0,14020.0,0.102304,0.056563,0.045742
film,11454.0,11534.0,0.084164,0.047407,0.036757
by,10274.0,9764.0,0.071248,0.042523,0.028725
debut,3824.0,6088.0,0.044424,0.015827,0.028597
short,4608.0,6228.0,0.045446,0.019072,0.026374
ep,3632.0,4854.0,0.035420,0.015033,0.020387
